# Test Notebook for Building Pipeline Using Module 'mpi_module'
The module 'mpi_module' was generated through command: 'az ml module init -n mpi_module'

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, Pipeline, dsl
import sys
from pathlib import Path
# The following line adds source directory to path. 
from mpi_module import mpi_module

## Prerequisites: Configure workspace & compute

 - Update config.json to reference your own workspace
 - Change the following aml_compute_target's value to reference an existing compute target

In [ ]:
workspace = Workspace.from_config(path = './config.json')
aml_compute_target = "YOUR_COMPUTE_TARGET"
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, aml_compute_target, sep='\n')

# Load module(s)
It can be done through one of the following:
- Module.from_func to convert a python function into module function
- Module.from_yaml to load a module function from a module yaml definition
- Module.load to load a module function from Azure ML workspace (for registered module)

In [ ]:
module_func = Module.from_func(workspace, mpi_module)help(module_func)

# Run module in local
Using module.run can execute module in local machine<br>
If use_docker=True, will pull image from azure and run module in container.<br>
If use_docker=False, will directly run module script.

In [ ]:
# create module
module = module_func().set_parameters(
    str_param='some_string'
).set_inputs(
    input_dir=str(Path('data') / 'mpi_module' / 'inputs' / 'input_dir')
)
module.run(use_docker=True)

# Create pipeline
This is done by calling module function(s) with the parameters/inputs/outputs the module supports

In [ ]:
# prepare dataset with the following code 

# from azureml.core import Dataset

# your_dataset = Dataset.get_by_name(name='YOUR_DATASET_NAME')


In [ ]:
@dsl.pipeline(name='mpi_module_pipeline', description='A sample pipeline uses mpi_module', default_compute_target=aml_compute_target)
def sample_pipeline():
    module1 = module_func().set_parameters(
    
    ).set_inputs(

    )
    module1.runsettings.configure(NodeCount = 1, ProcessCount=1)
    return module1.outputs

test_pipeline = sample_pipeline()

In [ ]:
test_pipeline.validate()

In [ ]:
run = test_pipeline.submit(
    experiment_name='mpi_module_experiment'
)
run.wait_for_completion()
run